In [1]:
import Gpib
import time
import matplotlib
import matplotlib.pyplot as plt

import ipywidgets as widgets

import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import numpy as np

%matplotlib inline


In [203]:
class ParamError(Exception):
    pass

class hp4142b(object):
    
    def __init__(self, inst):
        self.mode = "NORM"
        self.inst = inst
        self.reset()
        print(self.idn())
                
    def write(self, cmd):
        if not cmd.endswith("\n"):
            cmd = cmd+"\n"
        #print("CMD: %s" % cmd)
        return inst.write(cmd)

    def idn(self):
        self.write("*IDN?")
        try:
            return(inst.read().decode('ascii'))
        except:
            print(gpib_err(inst.ibsta()))
            return None   
    
    def average(self, n=0, auto=True):
        a=1
        if auto:
            a=0
            
        if(n>0):
            self.write("AV%d,%d" % (n, a))
        else:
            self.write("AV%d" % n)

    
    def status(self):
        self.write("*STB?")
        st = self.inst.read().decode('ascii')
        print("Status: %s %s" %(st,  statusdecode(int(st))))
        
    def nub(self):
        self.write("NUB?")
        return self.readresult()
    
    def units(self):
        self.write("UNT?")
        return self.readresult()
    
    def reset(self):
        self.write("*RST")
        self.write("FMT1,1")
    
    def voltage(self, ch, v=0.0, range=0, i_lim=0.001, lim_mode=0):
        return self.write("DV%d,%d,%+.3e,%+.3e,%d\n" % ( ch, range, v, i_lim, lim_mode ))
    
    def pulsed_v(self, ch, base=0.0, pulse=0.0, range=0, i_lim=0.001):
        self.mode="PULSED"
        return self.write("PDV%d,%d,%+.3e,%+.3e,%+.3e"%(ch, range, base, pulse, i_lim))
    
    def pulsed_i(self, ch, base=0.0, pulse=0.0, v_lim=1):
        self.mode="PULSED"
        return self.write("PDI%d,%d,%+.3e,%+.3e,%+.3e"%(ch, range, base, pulse, v_lim))
    
    def current(self, ch, i=0.0, range=0, v_lim=1, lim_mode=0):
        return self.write("DI%d,%d,%+.3e,%+.3e,%d\n"%(ch, range, i, v_lim, lim_mode))

    def sweep_i(self, ch, mode='lin', range=0, start=0, stop=1e-3, n=50, v_lim=1, p_lim=1):
        self.mode="SWEEP"
        mode_n=1;
        if mode=='log':
            mode_n=2
        if mode=='dlin':
            mode_n=3
        if mode=='dlog':
            mode_n=4
        return self.write("WI%d,%d,%d,%+.3e,%+.3e,%d,%+.3e,%+.3e\n" % (ch, mode_n, range, start, stop, n, v_lim, p_lim))

    def sweep_v(self, ch, mode='lin', range=0, start=0, stop=1, n=50, i_lim=0.001, p_lim=1):
        self.mode="SWEEP"
        mode_n=1;
        if mode=='log':
            mode_n=2
        if mode=='dlin':
            mode_n=3
        if mode=='dlog':
            mode_n=4
        return self.write("WV%d,%d,%d,%+.3e,%+.3e,%d,%+.3e,%+.3e\n" % (ch, mode_n, range, start, stop, n, i_lim, p_lim))

    def sweep_i_follow(self, ch, range=0, start=0.001, stop=0.001, v_lim=1, p_lim=1):
        return self.write("WSI%d,%d,%+.3e,%+.3e,%+.3e,%+.3e"%(ch, range, start, stop, v_lim, p_lim))
        
    def sweep_v_follow(self, ch, range=0, start=0, stop=1, i_lim=0.001, p_lim=1):
        return self.write("WSV%d,%d,%+.3e,%+.3e,%+.3e,%+.3e"%(ch, range, start, stop, i_lim, p_lim))
    
    def sweep_timing(self, hold=0, delay=0):
        return self.write("WT%.3e,%3e" % (hold, delay))
    
    def search_cfg(self, op="FBpos", meas="searchV", itime=5e-3):
        operations={ 'FBpos': 1,
              'FBneg': 2,
              'ramp_gt': 3,
              'ramp_lt': 4 }
        
        measurements = {'searchV': 1,
                        'searchI': 2,
                        'searchVsenseVI': 3,
                        'searchIsenseVI': 4 }
        
        opcode=operations[op] or None
        mcode = measurements[meas] or None
        
        self.write("ASM%d,%d,%f" % (opcode, mcode, itime))
        
    def search(self, ch, start=0, stop=1, rate=500, i_lim=1e-3):
        G=max(abs(start), abs(stop))
        D=abs(start-stop)
        self.write("ASV%d,%f,%f,%f,%f"%(ch, start, stop, rate, i_lim))
        
    def search_timing(self,hold=0.0, delay=0.0):
        self.write("AT%f,%f"%(hold, delay))
        
    def sense_i(self, ch, v=1, i_target=1e-4, i_lim=1e-3):
        self.write("AVI%d,%f,%f,%f"%(ch, v, i_target, i_lim))
    
    def sense_v(self, ch, i=1e-3, v_target=0.1, v_lim=1):
        self.write("AIV%d,%f,%f,%f"%(ch, i, v_target, v_lim))
    
        
    def connect(self,ch):
        cmd=""
        if ch:
            if type(ch)==type([]):
                cmd = "CN" + (",".join(list(map(str,ch)))) + "\n"
            else:
                cmd = "CN%s\n" % ch
        else:
            cmd="CN\n"
        self.write(cmd)

    def zero(self, ch=None):
        cmd=""
        if ch:
            if type(ch)==type([]):
                cmd = "DZ" + (",".join(list(map(str,ch)))) + "\n"
            else:
                cmd = "DZ%s\n" % ch
        else:
            cmd="DZ\n"
        self.write(cmd)
       
    def calibrate(self):
        self.write("CA")
        time.sleep(10)
        self.status()
        print("IBSTA: "+ gpib_err(inst.ibsta()))

        print("Calibrated")
    
    def disconnect(self,ch=None):
        cmd=""
        if ch:
            if type(ch)==type([]):
                cmd = "CL" + (",".join(list(map(str,ch)))) + "\n"
            else:
                cmd = "CL%s\n" % ch
        else:
            cmd="CL\n"
        self.write(cmd)
        
    def opstat(self):
        self.write("LOP?")
        codestring = self.readresult()
        codestring = codestring.replace("\n", "").replace("\r", "")
        codes = codestring[3:].split(",")

        codetotext = { '00': 'Not active',
               '01': 'V source',
               '02': 'I+ source',
               '03': 'I- source',
               '10': 'Both VS limit',
               '11': 'V compliance reached',
               '12': 'I+ compliance reached',
               '13': 'I- compliance reached',
               '20': 'Oscillating',
               '30': 'HVU not settled'
              }
        
        return list(map(lambda c: codetotext[c], codes))
        
        

        
    def readresult(self):
        readmore=True
        outdata=""
        ibread=""
        self._resdata = ""
    
        while readmore:
            try:
                ibread=inst.read().decode('ascii')
                if ibread.endswith("\n"):
                    readmore=False
            except:
            
                readmore=False
                
            outdata=outdata+ibread

        #print("IBSTA: "+ gpib_err(inst.ibsta()))
        self._resdata = outdata
        return outdata

    def xe(self):
        return self.write("XE")

    
def parseresult(res, channelnames):
    results = {}
    
    errors_reported = []

    items = res.split(",")
    for i in items:
        st=i[0]
        ch=channelnames[int((ord(i[1])-ord('A')+1)/2-1)]
        meas=i[2]
        v=float(i[3:])
        if(st in ('N', 'W', 'E')):
           resname="%s_%s"%(meas, ch)
           resarr=results.get(resname, [])
           resarr.append(v)
           results[resname] = resarr
        else:
            if not(st in errors_reported):
                print("MEAS_ERR: %s" % i)
                errors_reported.append(st)
    return results


hp4142n_status_bits= { 
    'Data ready': 0,
    'Wait': 1,
    'NotUsed': 2,
    'Interlock open': 3,
    'Set ready': 4,
    'Error': 5,
    'RQS': 6,
    'Shut down': 7
    }

gpib_err_bits={
        'DCAS' : 0,
        'DTAS' : 1,
        'LACS' : 2,
        'TACS' : 3,
        'ATN' : 4,
        'CIC' : 5,
        'REM' : 6,
        'LOK' : 7,
        'Complete' : 8,
        'EVENT' : 9,
        'SPOLL' : 10,
        'RQS' : 11,
        'SRQI' : 12,
        'END' : 13,
        'TIMO' : 14,
        'ERR' : 15
};

def gpib_err(bits):
    errs = []
    for err in gpib_err_bits:
        if bits & (2**gpib_err_bits[err]):
            errs.append(err)

    return " ".join(errs)

def statusdecode(bits):
    stat = []
    for err in hp4142n_status_bits:
        if bits & (2**hp4142n_status_bits[err]):
            stat.append(err)

    return " ".join(stat)

In [180]:
inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)

print(smu.units())
print(smu.opstat())


HEWLETT PACKARD,4142B,0,4.30

0,0;HP41420A,3;0,0;HP41420A,3;0,0;HP41420A,3;HP41425A,0;0,0

['Not active', 'Not active', 'Not active', 'Not active', 'Not active', 'Not active', 'Not active', 'Not active']


In [235]:
inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)

Ic_max=0.2

#2N3904 EBC pinout

S=0
G=2
D=4

B=2
C=4

channels=['G', 'D']

def meas_njfet(Vgs=0.0, Vds_max=10, Ids_max=0.1, Igs_max=1e-3):    
    smu.connect([G,D])
    smu.voltage(G, Vgs, i_lim=Igs_max)
  
    smu.sweep_v(D, start=0.01, stop=Vds_max, i_lim=Ids_max, mode='log')
    smu.write("MM2,2,4\n")

    smu.xe()
    smu.zero()
    smu.disconnect()

    r=smu.readresult()
    #print("RESULT: %s" % str(r))
    d=parseresult(r, channels)

    return d

def meas_vgs_ids(Vds=10, Vgs_min=-2, Vgs_max=0.5, Ids_max=10e-3):
    smu.connect([G,D])
    smu.voltage(D, Vds, i_lim=Ids_max)
  
    smu.sweep_v(G, start=Vgs_min, stop=Vgs_max, i_lim=1e-3)
    smu.write("MM2,2,4\n")

    smu.xe()
    smu.zero()
    smu.disconnect()

    r=smu.readresult()
    #print("RESULT: %s" % str(r))
    d=parseresult(r, channels)

    return d


def meas_idss(Vds=10.0, Vgs=0.0, Ids_max=0.1):
    smu.connect([G,D])
    smu.voltage(G, Vgs, i_lim=1e-3)
    smu.voltage(D, Vds, i_lim=Ids_max)
    
    smu.write("MM1,2,4")
    smu.xe()
    smu.disconnect()
    r=smu.readresult()
    d=parseresult(r, channels)
    return d['I_D'][0]

def meas_vgs_off(Ids=1e-6, Vds=1, Ids_max=0.15):
    smu.connect([D,G])
    smu.search(G, start=-2, stop=1, rate=10, i_lim=100e-6)
    smu.sense_i(D, v=Vds, i_target=Ids, i_lim=1.2*Ids) 
    smu.search_timing(hold=10e-3, delay=10e-3)
    smu.search_cfg(op='FBpos', meas="searchVsenseVI") #ASM 1,4,5e-4
    #smu.search_cfg(op='ramp_gt', meas="searchVsenseVI", itime=1e-3) #ASM 1,4,5e-4
    
    smu.write("MM6")
    smu.xe()
    #time.sleep(2)
    #print(smu.opstat())
    smu.zero()
    smu.disconnect()
    r=smu.readresult()
    #print(r)
    d=parseresult(r, channels)
    return d['V_G'][0]


HEWLETT PACKARD,4142B,0,4.30



In [227]:
#smu.calibrate()
#smu.average(10)
print(meas_idss())
print(meas_vgs_off(Ids=10e-6, Vds=10))
print(meas_vgs_ids())

0.0039006
-0.372
MEAS_ERR: TBI-07.2786E-09
MEAS_ERR: CDI+09.9994E-03
MEAS_ERR: VBI+199.999E+99
{'I_D': [1.3178e-07, 2.8228e-07, 5.8294e-07, 1.1686e-06, 2.3408e-06, 4.5676e-06, 8.5932e-06, 1.56e-05, 2.7342e-05, 4.655e-05, 7.7228e-05, 0.00012162, 0.00018666, 0.00027504, 0.00039274, 0.00054628, 0.0007365, 0.00096618, 0.0012302, 0.0015402, 0.001889, 0.0022708, 0.0026992, 0.003161, 0.003661, 0.0041832, 0.0047318, 0.005312, 0.0059286, 0.0065728, 0.0072384, 0.0079254, 0.0086306, 0.0093664], 'V_G': [-0.5, -0.4796, -0.4592, -0.4388, -0.4184, -0.398, -0.3776, -0.3572, -0.3368, -0.3164, -0.296, -0.2756, -0.2552, -0.2347, -0.2143, -0.1939, -0.1735, -0.1531, -0.1327, -0.1123, -0.0919, -0.0715, -0.0511, -0.0307, -0.0103, 0.0102, 0.0306, 0.051, 0.0714, 0.0918, 0.1122, 0.1326, 0.153, 0.1734, 0.1938], 'I_G': [2.5462e-09, 6.0792e-09, 8.492e-09, -3.8166e-09, 2.1838e-09, 6.1216e-09, 8.519e-09, 7.7474e-09, -3.6028e-09, -7.9666e-09, -4.3548e-09, 2.6684e-09, -6.8916e-09, -2.6212e-09, -7.886e-09, -5.152e-09, 

In [243]:
#Measure n JFET operating curves
data = []
smu = hp4142b(inst)

results = []

for vgs in np.arange(-0.5, 0.5, 0.1):
    d=meas_njfet(Vgs=float(vgs), Vds_max=20)
    print(vgs)
    ids=np.array(d['I_D'])
    igs=np.array(d['I_G'])
    vds=np.array(d['V_D'])
    results.append ( { 'Ids': ids,
                        'Igs': igs,
                        'Vds': vds,
                        'Vgs': vgs,
                      'part': '2SK170GR',
                     't': time.time() } )

    data.append( trace )
    
print("READY.")


HEWLETT PACKARD,4142B,0,4.30

-0.5
-0.4
-0.30000000000000004
-0.20000000000000007
-0.10000000000000009
-1.1102230246251565e-16
0.09999999999999987
0.19999999999999984
0.2999999999999998
0.3999999999999998
READY.


In [244]:
data= []

for r in results:
    trace =  go.Scatter(x=r['Vds'], y=1e3*r['Ids'], name='Vgs=%.2f' % (r['Vgs']))
    data.append(trace)

layout = go.Layout(
    title='2SK170',
    xaxis=dict(
        title='V_ds(mA)',
        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='I_ds(mA)', 
        #range=[0, 300],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout })

In [239]:
#Measure n JFET operating curves
data = []
smu = hp4142b(inst)

results = []

for vds in [5, 10, 15]:
    d=meas_vgs_ids(Vds=float(vds), Ids_max=30e-3, Vgs_min=-1)
    print(vds)
    ids=np.array(d['I_D'])
    igs=np.array(d['I_G'])
    vgs=np.array(d['V_G'])
    results.append ( { 'Ids': ids,
                        'Igs': igs,
                        'Vds': vds,
                        'Vgs': vgs,
                      'part': '2SK170GR',
                     't': time.time() } )

    data.append( trace )
    
print("READY.")

HEWLETT PACKARD,4142B,0,4.30

5
10
15
READY.


In [241]:
#print(results)
data= []

for r in results:
    trace =  go.Scatter(x=r['Vgs'], y=1e3*r['Ids'], name='Vds=%.2f' % (r['Vds']))
    data.append(trace)

layout = go.Layout(
    title='2SK170',
    xaxis=dict(
        title='V_gs(V)',
        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='I_ds(mA)', 
        #range=[0, 300],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout })

HEWLETT PACKARD,4142B,0,4.30

NBV-07.6784E+00,NDI-02.3624E-09

n=0 charge=0.023480 hfe=168.952703 Vebo=-7.678400
NBV-07.6780E+00,NDI+04.5246E-09

n=1 charge=0.023490 hfe=168.952703 Vebo=-7.678000
NBV-07.6780E+00,NDI-01.6934E-09

n=2 charge=0.023500 hfe=168.838623 Vebo=-7.678000
NBV-07.6784E+00,NDI+04.7078E-09

n=3 charge=0.023510 hfe=168.895643 Vebo=-7.678400
NBV-07.6788E+00,NDI+01.6654E-09

n=4 charge=0.023520 hfe=168.994602 Vebo=-7.678800
NBV-07.6788E+00,NDI-03.0128E-09

n=5 charge=0.023530 hfe=169.290541 Vebo=-7.678800
NBV-07.6804E+00,NDI+02.0574E-09

n=6 charge=0.023540 hfe=169.498984 Vebo=-7.680400
NBV-07.6804E+00,NDI-05.7532E-09

n=7 charge=0.023550 hfe=169.170897 Vebo=-7.680400
NBV-07.6800E+00,NDI+01.8598E-09

n=8 charge=0.023560 hfe=169.020270 Vebo=-7.680000
NBV-07.6772E+00,NDI+05.4376E-09

n=9 charge=0.023570 hfe=169.155405 Vebo=-7.677200
NBV-07.6752E+00,NDI-05.6244E-09

n=10 charge=0.023580 hfe=168.796484 Vebo=-7.675200
NBV-07.6804E+00,NDI+05.2652E-09

n=11 charge=0.023590 hf

NBV-07.6780E+00,NDI+04.0954E-09

n=109 charge=0.024570 hfe=168.750000 Vebo=-7.678000
NBV-07.6780E+00,NDI-04.7934E-09

n=110 charge=0.024580 hfe=168.507915 Vebo=-7.678000
NBV-07.6740E+00,NDI+02.4588E-09

n=111 charge=0.024590 hfe=168.790024 Vebo=-7.674000
NBV-07.6784E+00,NDI+06.0028E-09

n=112 charge=0.024600 hfe=168.851351 Vebo=-7.678400
NBV-07.6796E+00,NDI+06.1060E-09

n=113 charge=0.024610 hfe=168.817568 Vebo=-7.679600
NBV-07.6788E+00,NDI+06.3484E-09

n=114 charge=0.024620 hfe=169.108710 Vebo=-7.678800
NBV-07.6784E+00,NDI+02.7778E-09

n=115 charge=0.024630 hfe=168.823728 Vebo=-7.678400
NBV-07.6788E+00,NDI-01.5836E-09

n=116 charge=0.024640 hfe=168.952703 Vebo=-7.678800
NBV-07.6796E+00,NDI+04.9608E-09

n=117 charge=0.024650 hfe=168.621503 Vebo=-7.679600
NBV-07.6820E+00,NDI+05.5982E-09

n=118 charge=0.024660 hfe=168.851351 Vebo=-7.682000
NBV-07.6780E+00,NDI+06.6750E-09

n=119 charge=0.024670 hfe=169.009801 Vebo=-7.678000
NBV-07.6780E+00,NDI+06.0790E-09

n=120 charge=0.024680 hfe=169.15

NBV-07.6804E+00,NDI-06.1018E-09

n=206 charge=0.025540 hfe=168.461797 Vebo=-7.680400
NBV-07.6788E+00,NDI-05.1818E-09

n=207 charge=0.025550 hfe=168.916639 Vebo=-7.678800
NBV-07.6796E+00,NDI+05.7734E-09

n=208 charge=0.025560 hfe=168.667791 Vebo=-7.679600
NBV-07.6804E+00,NDI+06.8210E-09

n=209 charge=0.025570 hfe=168.781640 Vebo=-7.680400
NBV-07.6796E+00,NDI+04.0428E-09

n=210 charge=0.025580 hfe=168.676322 Vebo=-7.679600
NBV-07.6812E+00,NDI-05.6934E-09

n=211 charge=0.025590 hfe=168.756320 Vebo=-7.681200
NBV-07.6780E+00,NDI-03.8862E-09

n=212 charge=0.025600 hfe=168.710003 Vebo=-7.678000
NBV-07.6796E+00,NDI-03.7404E-09

n=213 charge=0.025610 hfe=169.074949 Vebo=-7.679600
NBV-07.6820E+00,NDI-05.6790E-09

n=214 charge=0.025620 hfe=168.507915 Vebo=-7.682000
NBV-07.6812E+00,NDI-02.2530E-09

n=215 charge=0.025630 hfe=168.994602 Vebo=-7.681200
NBV-07.6820E+00,NDI-02.0060E-09

n=216 charge=0.025640 hfe=168.474234 Vebo=-7.682000
NBV-07.6824E+00,NDI-05.5094E-09

n=217 charge=0.025650 hfe=168.77

NBV-07.6812E+00,NDI+02.5058E-09

n=311 charge=0.026590 hfe=168.621503 Vebo=-7.681200
NBV-07.6788E+00,NDI-05.7844E-09

n=312 charge=0.026600 hfe=168.771101 Vebo=-7.678800
NBV-07.6792E+00,NDI-05.2176E-09

n=313 charge=0.026610 hfe=168.642641 Vebo=-7.679200
NBV-07.6796E+00,NDI-02.2478E-09

n=314 charge=0.026620 hfe=168.575278 Vebo=-7.679600
NBV-07.6820E+00,NDI+03.9678E-09

n=315 charge=0.026630 hfe=168.474234 Vebo=-7.682000
NBV-07.6828E+00,NDI-03.5472E-09

n=316 charge=0.026640 hfe=168.621503 Vebo=-7.682800
NBV-07.6784E+00,NDI+03.1036E-09

n=317 charge=0.026650 hfe=168.688911 Vebo=-7.678400
NBV-07.6804E+00,NDI+05.3578E-09

n=318 charge=0.026660 hfe=168.849139 Vebo=-7.680400
NBV-07.6812E+00,NDI-02.9098E-09

n=319 charge=0.026670 hfe=168.760554 Vebo=-7.681200
NBV-07.6812E+00,NDI-03.1578E-09

n=320 charge=0.026680 hfe=168.927126 Vebo=-7.681200
NBV-07.6780E+00,NDI-06.0696E-09

n=321 charge=0.026690 hfe=168.539704 Vebo=-7.678000
NBV-07.6804E+00,NDI-03.3466E-09

n=322 charge=0.026700 hfe=168.58

NBV-07.6816E+00,NDI-05.6632E-09

n=408 charge=0.027560 hfe=168.539704 Vebo=-7.681600
NBV-07.6820E+00,NDI-06.0470E-09

n=409 charge=0.027570 hfe=168.022865 Vebo=-7.682000
NBV-07.6820E+00,NDI-05.7202E-09

n=410 charge=0.027580 hfe=168.596432 Vebo=-7.682000
NBV-07.6828E+00,NDI-04.7462E-09

n=411 charge=0.027590 hfe=168.451178 Vebo=-7.682800
NBV-07.6816E+00,NDI+03.1962E-09

n=412 charge=0.027600 hfe=168.663750 Vebo=-7.681600
NBV-07.6812E+00,NDI+05.7252E-09

n=413 charge=0.027610 hfe=168.653199 Vebo=-7.681200
NBV-07.6836E+00,NDI-04.7652E-09

n=414 charge=0.027620 hfe=168.564690 Vebo=-7.683600
NBV-07.6836E+00,NDI+06.3350E-09

n=415 charge=0.027630 hfe=168.518519 Vebo=-7.683600
NBV-07.6788E+00,NDI+03.0030E-09

n=416 charge=0.027640 hfe=168.596432 Vebo=-7.678800
NBV-07.6828E+00,NDI+04.0294E-09

n=417 charge=0.027650 hfe=168.327163 Vebo=-7.682800
NBV-07.6796E+00,NDI+02.8022E-09

n=418 charge=0.027660 hfe=168.724696 Vebo=-7.679600
NBV-07.6836E+00,NDI-03.1992E-09

n=419 charge=0.027670 hfe=168.82

...

NBV-07.6936E+00,NDI+05.9084E-09

n=2458 charge=0.048060 hfe=166.700000 Vebo=-7.693600
NBV-07.6908E+00,NDI+05.9568E-09

n=2459 charge=0.048070 hfe=166.433333 Vebo=-7.690800
NBV-07.6940E+00,NDI+03.2452E-09

n=2460 charge=0.048080 hfe=166.755585 Vebo=-7.694000
NBV-07.6940E+00,NDI-02.2546E-09

n=2461 charge=0.048090 hfe=166.611240 Vebo=-7.694000
NBV-07.6924E+00,NDI+01.5298E-09

n=2462 charge=0.048100 hfe=166.755408 Vebo=-7.692400
NBV-07.6936E+00,NDI+05.4752E-09

n=2463 charge=0.048110 hfe=166.966667 Vebo=-7.693600
NBV-07.6924E+00,NDI-03.4406E-09

n=2464 charge=0.048120 hfe=166.900000 Vebo=-7.692400
NBV-07.6952E+00,NDI-01.6960E-09

n=2465 charge=0.048130 hfe=166.477841 Vebo=-7.695200
NBV-07.6952E+00,NDI-02.4150E-09

n=2466 charge=0.048140 hfe=166.577985 Vebo=-7.695200
NBV-07.6948E+00,NDI+06.4062E-09

n=2467 charge=0.048150 hfe=166.555740 Vebo=-7.694800
NBV-07.6952E+00,NDI+03.1986E-09

n=2468 charge=0.048160 hfe=166.744341 Vebo=-7.695200
NBV-07.6956E+00,NDI-05.5700E-09

n=2469 charge=0.04817

NBV-07.6952E+00,NDI-05.7258E-09

n=2554 charge=0.049020 hfe=166.788930 Vebo=-7.695200
NBV-07.6940E+00,NDI+05.2610E-09

n=2555 charge=0.049030 hfe=166.500000 Vebo=-7.694000
NBV-07.6936E+00,NDI-03.4386E-09

n=2556 charge=0.049040 hfe=166.533467 Vebo=-7.693600
NBV-07.6968E+00,NDI+06.1676E-09

n=2557 charge=0.049050 hfe=167.033333 Vebo=-7.696800
NBV-07.6948E+00,NDI-05.3556E-09

n=2558 charge=0.049060 hfe=166.733267 Vebo=-7.694800
NBV-07.6936E+00,NDI+04.1326E-09

n=2559 charge=0.049070 hfe=166.566667 Vebo=-7.693600
NBV-07.6968E+00,NDI-05.1800E-09

n=2560 charge=0.049080 hfe=166.755585 Vebo=-7.696800
NBV-07.6932E+00,NDI-02.5010E-09

n=2561 charge=0.049090 hfe=166.966667 Vebo=-7.693200
NBV-07.6932E+00,NDI+03.9710E-09

n=2562 charge=0.049100 hfe=166.588863 Vebo=-7.693200
NBV-07.6968E+00,NDI+03.7788E-09

n=2563 charge=0.049110 hfe=166.311939 Vebo=-7.696800
NBV-07.6948E+00,NDI-01.8942E-09

n=2564 charge=0.049120 hfe=166.888963 Vebo=-7.694800
NBV-07.6940E+00,NDI+01.3684E-09

n=2565 charge=0.04913

NBV-07.6952E+00,NDI+02.2124E-09

n=2660 charge=0.050080 hfe=166.966667 Vebo=-7.695200
NBV-07.6936E+00,NDI+06.4116E-09

n=2661 charge=0.050090 hfe=166.700000 Vebo=-7.693600
NBV-07.6936E+00,NDI+03.4368E-09

n=2662 charge=0.050100 hfe=166.744341 Vebo=-7.693600
NBV-07.6920E+00,NDI-02.4266E-09

n=2663 charge=0.050110 hfe=166.566667 Vebo=-7.692000
NBV-07.6964E+00,NDI-04.5122E-09

n=2664 charge=0.050120 hfe=166.555740 Vebo=-7.696400
NBV-07.6972E+00,NDI-05.1172E-09

n=2665 charge=0.050130 hfe=166.400798 Vebo=-7.697200
NBV-07.6952E+00,NDI-04.0602E-09

n=2666 charge=0.050140 hfe=166.655563 Vebo=-7.695200
NBV-07.6932E+00,NDI+05.9946E-09

n=2667 charge=0.050150 hfe=166.900000 Vebo=-7.693200
NBV-07.6952E+00,NDI+06.2124E-09

n=2668 charge=0.050160 hfe=166.633333 Vebo=-7.695200
NBV-07.6936E+00,NDI+04.2012E-09

n=2669 charge=0.050170 hfe=166.245427 Vebo=-7.693600
NBV-07.6936E+00,NDI+06.5804E-09

n=2670 charge=0.050180 hfe=166.466866 Vebo=-7.693600
NBV-07.6948E+00,NDI-01.2594E-09

n=2671 charge=0.05019

NBV-07.6956E+00,NDI+02.8044E-09

n=2756 charge=0.051040 hfe=166.500167 Vebo=-7.695600
NBV-07.6948E+00,NDI+04.6048E-09

n=2757 charge=0.051050 hfe=166.577985 Vebo=-7.694800
NBV-07.6956E+00,NDI+02.0556E-09

n=2758 charge=0.051060 hfe=166.378450 Vebo=-7.695600
NBV-07.6936E+00,NDI-05.4964E-09

n=2759 charge=0.051070 hfe=166.400798 Vebo=-7.693600
NBV-07.6948E+00,NDI+02.8124E-09

n=2760 charge=0.051080 hfe=166.810919 Vebo=-7.694800
NBV-07.6936E+00,NDI+05.0476E-09

n=2761 charge=0.051090 hfe=166.688852 Vebo=-7.693600
NBV-07.6952E+00,NDI+04.6184E-09

n=2762 charge=0.051100 hfe=166.478029 Vebo=-7.695200
NBV-07.6964E+00,NDI-05.4858E-09

n=2763 charge=0.051110 hfe=166.323138 Vebo=-7.696400
NBV-07.6936E+00,NDI-03.9116E-09

n=2764 charge=0.051120 hfe=166.400266 Vebo=-7.693600
NBV-07.6936E+00,NDI+05.4646E-09

n=2765 charge=0.051130 hfe=166.400266 Vebo=-7.693600
NBV-07.6916E+00,NDI+05.3212E-09

n=2766 charge=0.051140 hfe=166.356692 Vebo=-7.691600
NBV-07.6920E+00,NDI+06.1062E-09

n=2767 charge=0.05115

NBV-07.6936E+00,NDI+06.2642E-09

n=2852 charge=0.052000 hfe=166.356692 Vebo=-7.693600
NBV-07.6968E+00,NDI+05.5606E-09

n=2853 charge=0.052010 hfe=166.422629 Vebo=-7.696800
NBV-07.6936E+00,NDI+03.6408E-09

n=2854 charge=0.052020 hfe=166.334330 Vebo=-7.693600
NBV-07.6956E+00,NDI+03.0702E-09

n=2855 charge=0.052030 hfe=166.356692 Vebo=-7.695600
NBV-07.6936E+00,NDI+03.4148E-09

n=2856 charge=0.052040 hfe=166.533467 Vebo=-7.693600
NBV-07.6940E+00,NDI+05.2534E-09

n=2857 charge=0.052050 hfe=166.677763 Vebo=-7.694000
NBV-07.6940E+00,NDI+06.3978E-09

n=2858 charge=0.052060 hfe=166.091000 Vebo=-7.694000
NBV-07.6948E+00,NDI+05.3464E-09

n=2859 charge=0.052070 hfe=166.434031 Vebo=-7.694800
NBV-07.6948E+00,NDI+02.8580E-09

n=2860 charge=0.052080 hfe=166.422629 Vebo=-7.694800
NBV-07.6948E+00,NDI-01.7262E-09

n=2861 charge=0.052090 hfe=166.201396 Vebo=-7.694800
NBV-07.6980E+00,NDI+06.4292E-09

n=2862 charge=0.052100 hfe=166.744341 Vebo=-7.698000
NBV-07.6972E+00,NDI+04.1008E-09

n=2863 charge=0.05211

In [34]:
layout = go.Layout(
    title='2N3904 Vebo damage at %.0f uA I_eb' % (iebo*1e6) ,
    xaxis=dict(
        title='Charge, A*s',
        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='hFE', 
        range=[0, 300],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)

trace =  go.Scatter(x=acharge, y=ahfe, name='hFE vs reverse EBO charge %f uA' % (1e6*iebo))
data=[trace]
iplot({ 'data': data, "layout": layout })

In [27]:
layout = go.Layout(
    title='2N3904 Vebo damage at 100/1000 uA I_eb' ,
    xaxis=dict(
        title='Charge, A*s',
        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='Veb', 
        range=[-8, -7],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)

trace =  go.Scatter(x=acharge, y=avebo, name='V_eb vs reverse EBO charge %f uA' % (1e6*iebo))
data=[trace]
iplot({ 'data': data, "layout": layout })

In [149]:
#Measure BE reverse voltage - can destroy transistor
#print(vbceo_npn(Iebo=1e-6))


['Not active', 'I- source', 'Not active', 'V source', 'Not active', 'Not active', 'Not active', 'Not active']
-7.8744


In [32]:

charge

0.023469999999999214

In [8]:
ic=np.array(vce_sat['I_C'])
vc=np.array(vce_sat['V_B'])
trace =  go.Scatter(x=1e3*ic, y=vb, name='V_BE sat vs I_C')

data=[trace]

layout = go.Layout(
    title='2N3904 VCE sat.',
    xaxis=dict(
        title='I_C(mA)',
        type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='VBE sat.',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout }) 
    

In [133]:
channels=['X','Y','A']
A=6

r=None

inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)
#smu.calibrate()

def meas_diode():
    smu.connect([A])
    smu.sweep_i(A, start=1e-8, stop=100e-3, v_lim=3, mode='log', n=50)
    smu.write("MM2,6\n")

    smu.xe()
    smu.zero()
    smu.disconnect()

    r=smu.readresult()
    #print(r)

    d=parseresult(r, channels)

    return d

def meas_diode_rev(): 
    smu.connect([A])
    smu.sweep_v(A, start=-40, stop=-0.1, i_lim=0.001, n=40)
    smu.write("MM2,6\n")
    smu.sweep_timing(hold=0.05, delay=0.05)
    smu.xe()
#    time.sleep(15)
    smu.zero()
    smu.disconnect()
    smu.status()

    r=smu.readresult()
    #print(r)
    d=parseresult(r, channels)

    return d


diode=meas_diode()
rev=meas_diode_rev()
print("Done")


HEWLETT PACKARD,4142B,0,4.30

IBSTA: Complete END
Status: 0
 
IBSTA: Complete END
Done


In [118]:

tracerev =  go.Scatter(x=np.array(rev['V_A']), y=np.array(rev['I_A']), name='Reverse')
tracefwd = go.Scatter(x=np.array(diode['V_A']), y=np.array(diode['I_A']), name="Forward")

data=[tracerev, tracefwd]
#data= [tracerev]
layout = go.Layout(
    title='Red LED diode forward drop',
    xaxis=dict(
        title='V',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='I',
        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout }) 
    

In [48]:
rev['I_A']

[-1.06814e-08,
 -1.00198e-08,
 -1.7704e-09,
 -8.3544e-09,
 -5.6358e-09,
 -1.9352e-09,
 -8.8592e-09,
 -1.2968e-09,
 -3.3972e-09,
 -6.9222e-09,
 1.3822e-09,
 -5.224e-09,
 -2.8544e-09,
 6.678e-10,
 -2.4192e-09,
 1.7328e-09,
 -5.206e-09,
 2.017e-09,
 -2.0618e-09,
 3.3278e-09]

In [145]:
inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)

smu.connect([A])
smu.current(A, 1e-3, v_lim=5)
#smu.write("MM1,6\n")
#smu.xe()
time.sleep(4)
smu.current(A, 10e-3, v_lim=5)
time.sleep(2)
smu.disconnect()

HEWLETT PACKARD,4142B,0,4.30

